In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
from Hourbid import *
from Fibid import *
from blockbid import *


In [2]:
HourBid()[1]

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,12,1000,1,S,164967.0,2000.00,1,NaN
1,12,999,1,S,164998.0,1996.43,1,NaN
2,12,998,1,S,165014.0,1994.60,1,NaN
3,12,997,1,S,165025.0,1993.33,1,NaN
4,12,996,1,S,165050.0,1990.44,1,NaN
...,...,...,...,...,...,...,...,...
23995,242,5,24,S,197374.0,6.99,1,NaN
23996,242,4,24,S,197957.0,4.23,1,NaN
23997,242,3,24,S,198338.0,2.80,1,NaN
23998,242,2,24,S,199205.0,0.64,1,NaN


In [3]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

In [4]:
np.shape(BD_bids)

(110, 8)

In [ ]:
L_D = len(BD_bids)
L_S = len(BS_bids)

In [7]:
BD_bids

,ID,OrderID,Hour,BitType,Quantity,price,Duration,LinkID
0,1215,1,6,B,845.0,589.00,9,1218.0
1,1210,1,5,B,367.0,606.77,15,NaN
2,1218,1,1,B,526.0,592.34,23,NaN
3,1110,1,6,B,888.0,614.76,6,NaN
4,1231,1,11,B,335.0,808.14,3,NaN
...,...,...,...,...,...,...,...,...
105,1093,1,10,B,714.0,577.24,10,NaN
106,1203,1,5,B,94.0,586.87,18,NaN
107,1202,1,2,B,551.0,506.76,20,NaN
108,1208,1,5,B,484.0,623.42,18,NaN


In [8]:
np.where(BD_bids['ID']==1232)[0][0]

26

In [9]:
NoneChild_D=[]
Child_D = []
Parent_D=[]
NoneChild_S=[]
Child_S=[]
Parent_S=[]
for i in range(L_D):
    if BD_bids.isnull()['LinkID'][i] == True:
        NoneChild_D.append(i)
    else:
        Parent_D.append(np.where(BD_bids['ID']==BD_bids['LinkID'][i])[0][0])
        Child_D.append(i)
for i in range(L_S):
    if BS_bids.isnull()['LinkID'][i] == True:
        NoneChild_S.append(i)
    else:
        Parent_S.append(np.where(BS_bids['ID']==BS_bids['LinkID'][i])[0][0])
        Child_S.append(i)

In [10]:
Delta_S = np.zeros([L_S,24])


Delta_D = np.zeros([L_D,24])
BD_bids  = Blockbid()[1]
for i in range (L_D):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=25:
        Delta_D[i,BD_bids['Hour'][i]-1:(BD_bids['Hour'][i]+BD_bids['Duration'][i])-1] = 1

BS_bids = Blockbid()[0]
for i in range (L_S):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=25:
        Delta_S[i,BS_bids['Hour'][i]-1:(BS_bids['Hour'][i]+BS_bids['Duration'][i])-1] = 1

In [11]:
def find_F(Del,leng,bids):
    Res = [[] for i in range(24)]
    Res_max = [[] for i in range(24)]
    Res_min = [[] for i in range(24)]
    for i in range (24):
        for j in range(leng):
            if Del[j,i] == 1:
                Res[i].append(bids['price'][j])
    for i in range(24):
        Res_max[i] = np.max(Res[i])
        Res_min[i] = np.min(Res[i])
    return(Res_max,Res_min)

FD_min = np.min(FD_bids['price'])
FD_max = np.max(FD_bids['price'])
FS_min = np.min(FS_bids['price'])
FS_max = np.max(FS_bids['price'])

In [12]:
HD_bids['price'][HIndex_D[1][1]]

0.0

In [13]:
HIndex_D[1][1]

1999

In [14]:
DMax_Res = [[] for i in range(24)]
Dm_Res = [[] for i in range(24)]
SMax_Res = [[] for i in range(24)]
Sm_Res = [[] for i in range(24)]

for i in range(24):
    DMax_Res[i].append(HD_bids['price'][HIndex_D[i][0]])
    DMax_Res[i].append(find_F(Delta_D,L_D,BD_bids)[0][i])
    DMax_Res[i].append(FD_max)
    
for i in range(24):
    SMax_Res[i].append(HS_bids['price'][HIndex_S[i][1]])
    SMax_Res[i].append(find_F(Delta_S,L_S,BS_bids)[0][i])
    SMax_Res[i].append(FS_max)
    
for i in range(24):
    Dm_Res[i].append(HD_bids['price'][HIndex_D[i][1]])
    Dm_Res[i].append(find_F(Delta_D,L_D,BD_bids)[1][i])
    Dm_Res[i].append(FD_min)
    
for i in range(24):
    Sm_Res[i].append(HS_bids['price'][HIndex_S[i][0]])
    Sm_Res[i].append(find_F(Delta_S,L_S,BS_bids)[1][i])
    Sm_Res[i].append(FS_min)

In [15]:
def findmin(num1,num2,num3):
    
    return(min([num1,num2,num3]))

def findmax(num1,num2,num3):

    return(max([num1,num2,num3]))


def put_into_list(maximums,minimums):
    F_min = []
    F_max = []
    for x in range(24):
        F_min.append(findmin(minimums[x][0],minimums[x][1],minimums[x][2]))
        F_max.append(findmax(maximums[x][0],maximums[x][1],maximums[x][2]))

    return(F_min,F_max)

In [16]:
E = Model("Energy")
E.params.NonConvex =2
E.update()
J = np.arange(1,find_largest(HIndex_D)+1)
K = np.arange(1,find_largest(HIndex_S)+1)
H = np.arange(1,25)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-02
Set parameter NonConvex to value 2


In [17]:
L_FD = len(Fibid()[1])
L_FS = len(Fibid()[0])
L_FS

29

In [18]:
xD = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xD')
xS = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.CONTINUOUS, name='xS')
w_D = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.BINARY,name ='W_D')
w_S = E.addVars(H,J,ub=1,lb=0,  vtype=GRB.BINARY,name ='W_S')
yD = E.addVars(L_D,ub=1,lb=0,  vtype=GRB.BINARY,name ='yD')
yS = E.addVars(L_S,ub=1,lb=0,  vtype=GRB.BINARY,name ='yS')
F_z = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_Z')
# F_zD = E.addVars(H,lb = 0,vtype=GRB.CONTINUOUS, name='F_ZD')
V_S = E.addVars(H,L_FS,lb=0,vtype=GRB.BINARY,name = 'V_S')
V_D = E.addVars(H,L_FD,lb=0,vtype=GRB.BINARY,name = 'V_D')
# E.setParam('MIPGap', 0.1)
# E.setParam('Timelimit', 7200)

In [19]:
E.setObjective(sum(0.5*(2*Pair(j,h,HD_bids,'DP')[0]+xD[h,j]*(Pair(j,h,HD_bids,'DP')[1]-Pair(j,h,HD_bids,'DP')[0]))*xD[h,j]*(Pair(j,h,HD_bids,'DQ')[1]-Pair(j,h,HD_bids,'DQ')[0])for j in J for h in H)
                -sum(0.5*(2*Pair(j,h,HS_bids,'SP')[0]+xS[h,j]*(Pair(j,h,HS_bids,'SP')[1]-Pair(j,h,HS_bids,'SP')[0]))*xS[h,j]*(Pair(j,h,HS_bids,'SQ')[1]-Pair(j,h,HS_bids,'SQ')[0])for j in J for h in H)
                +sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*yD[i] for i in range(L_D)) - sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*yS[i] for i in range(L_S))
                +sum(FD_bids['Quantity'][i]*FD_bids['price'][i] for i in range(L_FD))*sum(V_D[h,i]for i in range(L_FD) for h in H)-sum(FS_bids['Quantity'][i]*FS_bids['price'][i] for i in range(L_FS))*sum(V_S[h,i]for i in range(L_FS) for h in H)
                +sum(Pair(1,h,HD_bids,'DQ')[0]-Pair(1,h,HD_bids,'DP')[0] for h in H)
                
,GRB.MAXIMIZE)

In [20]:
gamma = 1000000

        # (2)
E.addConstrs(-sum((Pair(j,h,HD_bids,'DQ')[1]-Pair(j,h,HD_bids,'DQ')[0]) * xD[h,j] for j in J ) +sum((Pair(j,h,HS_bids,'SQ')[1]-Pair(j,h,HS_bids,'SQ')[0])*xS[h,j] for j in J) -Pair(1,h,HD_bids,'DQ')[0] +Pair(1,h,HS_bids,'SQ')[0] 
                    +sum(Delta_S[i,h-1]*BS_bids['Quantity'][i]*yS[i] for i in range(L_S))  -sum(Delta_D[i,h-1]*BD_bids['Quantity'][i]*yD[i] for i in range(L_D))
                    +sum(FS_bids['Quantity'][i]*V_S[h,i]for i in range(L_FS))-sum(FD_bids['Quantity'][i]*V_D[h,i]for i in range(L_FD))
                    
                    ==0 for h in H) 
 


E.addConstrs(F_z[h] == Sm_Res[h-1][0]+sum((Pair(j,h,HS_bids,'SP')[1]-Pair(j,h,HS_bids,'SP')[0]) * xS[h,j] for j in J)for h in H)        
E.addConstrs(F_z[h] == DMax_Res[h-1][0]+sum((Pair(j,h,HD_bids,'DP')[1]-Pair(j,h,HD_bids,'DP')[0]) * xD[h,j] for j in J) for h in H)
# (8)

# E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_z[h] for h in H ) <= yS[j]*gamma for j in range(L_S))
# # (9)

# E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_z[h] for h in H) <= yD[j]*gamma for j in range(L_D))

E.addConstrs(-BS_bids['Duration'][j]* BS_bids['price'][j]+ sum(Delta_S[j,h-1]*F_z[h] for h in H ) <= yS[j]*gamma for j in NoneChild_S)
# (9)

E.addConstrs(BD_bids['Duration'][j]*BD_bids['price'][j]- sum(Delta_S[j,h-1]*F_z[h] for h in H) <= yD[j]*gamma for j in NoneChild_D)
        
        
    # E.addConstr(if Blockbid['LinkID'][j] == 'NaN': Blockbid['Duration'][j]*Blockbid['price'][j]+ sum(Delta_S[j,h]*F_z[h] for j in J) <= yd[j]*gamma)
        
# (3)(4)
# Demand
E.addConstrs(w_D[h,1]<=xD[h,1] for h in H)
E.addConstrs(xD[h,1] <=1 for h in H)
E.addConstrs(w_D[h,j]<=xD[h,j] for j in range(2,find_largest(HIndex_D)) for h in H)
E.addConstrs(xD[h,j]<=w_D[h,j-1] for j in range(2,find_largest(HIndex_D)) for h in H)
E.addConstrs(0<= xD[h,999] for h in H)
E.addConstrs(xD[h,999] <= w_D[h,998] for h in H)

# Supply
E.addConstrs(w_S[h,1]<=xS[h,1] for h in H)
E.addConstrs(xS[h,1] <=1 for h in H)
E.addConstrs(w_S[h,j]<=xS[h,j] for j in range(2,find_largest(HIndex_S)) for h in H)
E.addConstrs(xS[h,j]<=w_S[h,j-1] for j in range(2,find_largest(HIndex_S)) for h in H)
E.addConstrs(0<= xS[h,999] for h in H)
E.addConstrs(xS[h,999] <= w_S[h,998] for h in H)


E.addConstrs(sum(V_S[h,i] for h in H )<= 1 for i in range(L_FS) )
E.addConstrs(sum(V_D[h,i] for h in H )<= 1 for i in range(L_FD) )

E.addConstrs(F_z[h]-FS_bids['price'][i] <= gamma*sum(V_S[k,i] for k in H) for i in range(L_FS) for h in H)
E.addConstrs(FD_bids['price'][i]-F_z[h]<=gamma*sum(V_D[k,i] for k in H) for i in range(L_FD) for h in H)


E.addConstrs(yD[Child_D[i]]<=yD[Parent_D[i]] for i in range(len(Child_D)))
E.addConstrs(yS[Child_S[i]]<=yS[Parent_S[i]] for i in range(len(Child_S)))


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [21]:
import time
Inital =time.time()
E.optimize()
Final =time.time()-Inital

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97521 rows, 97423 columns and 326673 nonzeros
Model fingerprint: 0xea8e2de7
Model has 47869 quadratic objective terms
Variable types: 47976 continuous, 49447 integer (49447 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+06]
  Objective range  [3e+01, 2e+07]
  QObjective range [2e-02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Presolve removed 112 rows and 11 columns
Presolve time: 0.50s
Presolved: 97409 rows, 97412 columns, 299412 nonzeros
Presolved model has 47867 quadratic objective terms
Variable types: 47974 continuous, 49438 integer (49438 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   29884    2.4993836e+08   0.000000e+00   4.719525e+09      5s
   47959    2.3793223e+08   0.000000e+00   1.410425e+09     10s
   62660    2.3630754e

In [ ]:
import time
Inital =time.time()
E.optimize()
Final =time.time()-Inital

In [ ]:
E.getAttr('x',xD)

{(1, 1): 1.0,
 (1, 2): 1.0,
 (1, 3): 1.0,
 (1, 4): 1.0,
 (1, 5): 1.0,
 (1, 6): 1.0,
 (1, 7): 1.0,
 (1, 8): 1.0,
 (1, 9): 1.0,
 (1, 10): 1.0,
 (1, 11): 1.0,
 (1, 12): 1.0,
 (1, 13): 1.0,
 (1, 14): 1.0,
 (1, 15): 1.0,
 (1, 16): 1.0,
 (1, 17): 1.0,
 (1, 18): 1.0,
 (1, 19): 1.0,
 (1, 20): 1.0,
 (1, 21): 1.0,
 (1, 22): 1.0,
 (1, 23): 1.0,
 (1, 24): 1.0,
 (1, 25): 1.0,
 (1, 26): 1.0,
 (1, 27): 1.0,
 (1, 28): 1.0,
 (1, 29): 1.0,
 (1, 30): 1.0,
 (1, 31): 1.0,
 (1, 32): 1.0,
 (1, 33): 1.0,
 (1, 34): 1.0,
 (1, 35): 1.0,
 (1, 36): 1.0,
 (1, 37): 1.0,
 (1, 38): 1.0,
 (1, 39): 1.0,
 (1, 40): 1.0,
 (1, 41): 1.0,
 (1, 42): 1.0,
 (1, 43): 1.0,
 (1, 44): 1.0,
 (1, 45): 1.0,
 (1, 46): 1.0,
 (1, 47): 1.0,
 (1, 48): 1.0,
 (1, 49): 1.0,
 (1, 50): 1.0,
 (1, 51): 1.0,
 (1, 52): 1.0,
 (1, 53): 1.0,
 (1, 54): 1.0,
 (1, 55): 1.0,
 (1, 56): 1.0,
 (1, 57): 1.0,
 (1, 58): 1.0,
 (1, 59): 1.0,
 (1, 60): 1.0,
 (1, 61): 1.0,
 (1, 62): 1.0,
 (1, 63): 1.0,
 (1, 64): 1.0,
 (1, 65): 1.0,
 (1, 66): 1.0,
 (1, 67): 1.0,
 (1,

In [ ]:
Index = [v.VarName for v in xS.values()]
value = [v.x for v in xS.values()]
Solution_HS = [[]for i in range(len(J))]
for j in range(len(J)):
    for i in range(24):
        Solution_HS[j].append(value[len(J)*i+j])

Solution_HS = pd.DataFrame(Solution_HS)
Solution_HS
Solution_HS.to_csv('Result/HS.csv',index=False,header=False)

In [ ]:
Index = [v.VarName for v in xD.values()]
value = [v.x for v in xD.values()]
Solution_HD = [[]for i in range(len(J))]
for j in range(len(J)):
    for i in range(24):
        Solution_HD[j].append(value[len(J)*i+j])

Solution_HD = pd.DataFrame(Solution_HD)
Solution_HD.to_csv('Result/HD.csv',index=False,header=False)

In [ ]:
def Find_H_Break(path,DF,SD):
    
    Data = pd.read_csv(path,header = None)
    Res=[[] for i in range(24)]
    for i in range(24):
        for j in range(len(Data)):
            if Data.values[j][i] >= 0.00001 and Data.values[j][i] <= 0.999999:
                Res[i].append(j)
                Res[i].append(Data.values[j][i])
                Res[i].append(sum((Pair(j,i+1,DF,SD)[1]-Pair(j,i+1,DF,SD)[0]) * Data.values[j-1][i] for j in range(1,len(Data)+1)))
                
    return Res

In [ ]:
Find_H_Break(path='Result/HS.csv',DF=HS_bids,SD='SP')

[[64, 0.739115985377191, 128.14302093718842],
 [602, 0.6102312033497201, 1204.605699981795],
 [402, 0.0045871559632803, 803.199633027523],
 [166, 0.7952908903877036, 333.02947651160173],
 [314, 0.3487231098250253, 627.7718077745625],
 [200, 0.2051094890508827, 400.18328467153225],
 [528, 0.4557285954521859, 1057.644764202676],
 [260, 0.2809917355373545, 520.0699173553721],
 [255, 0.3220735785948151, 509.70259197324356],
 [176, 0.9022687609076324, 353.72214659685903],
 [213, 0.5343618513319514, 426.3088779803637],
 [338, 0.7002503477042338, 676.6216584144621],
 [144, 0.8079781962333917, 289.7789990089188],
 [254, 0.4175576814855899, 508.3067416994934],
 [390, 0.1386756238002625, 779.8077063339728],
 [245, 0.3884020422159172, 490.0365411872284],
 [365, 0.4918330308528302, 730.7255172413787],
 [194, 0.3848126513749544, 388.1889428693549],
 [384, 0.4196078431371967, 767.8392156862744],
 [348, 0.1759259259258189, 696.0062962962961],
 [396, 0.3996412556052405, 793.2331354260086],
 [53, 0.042

In [ ]:
2000+Find_H_Break(path='Result/HD.csv',DF=HD_bids,SD='DP')[0][2]


128.14302093719039

In [ ]:
Find_H_Break(path='Result/HD.csv',DF=HD_bids,SD='DP')

[[934, 0.5748421402459281, -1871.8569790628096],
 [396, 0.373202257418545, -795.3943000182051],
 [597, 0.3761467889907879, -1196.8003669724778],
 [832, 0.132586581323669, -1666.9705234884],
 [684, 0.7119763492260953, -1372.2281922254367],
 [798, 0.3973722627730014, -1599.8167153284655],
 [470, 0.2533450318050356, -942.3552357973234],
 [739, 0.0656689747600171, -1479.9300826446274],
 [743, 0.6175585284278423, -1490.2974080267552],
 [821, 0.6034031413614447, -1646.2778534031386],
 [785, 0.8246844319771126, -1573.6911220196346],
 [661, 0.0142420027813496, -1323.3783415855353],
 [853, 0.5792864222000219, -1710.2210009910789],
 [744, 0.7618560392897983, -1491.6932583005073],
 [609, 0.1513115802939983, -1220.1922936660267],
 [753, 0.6911529375904529, -1509.963458812771],
 [633, 0.1271551724132075, -1269.2744827586198],
 [804, 0.676307166263113, -1611.811057130644],
 [614, 0.7607843137253945, -1232.1607843137251],
 [650, 0.5629629629626206, -1303.993703703702],
 [601, 0.8133632286996721, -120